<!-- NOTEBOOK_METADATA source: "⚠️ Jupyter Notebook" title: "Trace the OpenAI Agents SDK with Langfuse" sidebarTitle: "OpenAI Agents" logo: "/images/integrations/openai_icon.svg" description: "Learn how to use Langfuse to monitor OpenAI Agents SDK to debug and evaluate your AI agents" category: "Integrations" -->

# Trace the OpenAI Agents SDK with Langfuse

This notebook demonstrates how to **integrate Langfuse** into your **OpenAI Agents** workflow to monitor, debug and evaluate your AI agents.

> **What is the OpenAI Agents SDK?**: The [OpenAI Agents SDK](https://openai.github.io/openai-agents-python/) is a lightweight, open-source framework that lets developers build AI agents and orchestrate multi-agent workflows. It provides building blocks—such as tools, handoffs, and guardrails to configure large language models with custom instructions and integrated tools. Its Python-first design supports dynamic instructions and function tools for rapid prototyping and integration with external systems.

> **What is Langfuse?**: [Langfuse](https://langfuse.com/) is an open-source observability platform for AI agents. It helps you visualize and monitor LLM calls, tool usage, cost, latency, and more.

## 1. Install Dependencies

Below we install the `openai-agents` library (the OpenAI Agents SDK), and the `pydantic-ai[logfire]` OpenTelemetry instrumentation.

In [1]:
%pip install openai-agents langfuse nest_asyncio "pydantic-ai[logfire]"

  Using cached pyperclip-1.11.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached authlib-1.6.5-py2.py3-none-any.whl.metadata (9.8 kB)
  Using cached cyclopts-4.3.0-py3-none-any.whl.metadata (12 kB)
  Using cached exceptiongroup-1.3.1-py3-none-any.whl.metadata (6.7 kB)
  Using cached jsonschema_path-0.3.4-py3-none-any.whl.metadata (4.3 kB)
  Using cached openapi_pydantic-0.5.1-py3-none-any.whl.metadata (10 kB)
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
  Using cached pathvalidate-3.3.1-py3-none-any.whl.metadata (12 kB)
  Using cached rich_rst-1.3.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached docutils-0.22.3-py3-none-any.whl.metadata (15 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow 2.13.2 requires protobuf<5,>=3.12.0, but you have protobuf 6.33.1 which is incompatible.
mlflow 2.13.2 requires pyarrow<16,>=4.0.0, but you have pyarrow 22.0.0 which is incompatible.


## 2. Configure Environment & Langfuse Credentials

Next, set up your Langfuse API keys. You can get these keys by signing up for a free [Langfuse Cloud](https://cloud.langfuse.com/) account or by [self-hosting Langfuse](https://langfuse.com/self-hosting). These environment variables are essential for the Langfuse client to authenticate and send data to your Langfuse project.

In [ ]:
from google.colab import userdata
import os
import base64


# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = userdata.get('LANGFUSE_PUBLIC_KEY')
os.environ["LANGFUSE_SECRET_KEY"] = userdata.get('LANGFUSE_SECRET_KEY')
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

# Build Basic Auth header.
#LANGFUSE_AUTH = base64.b64encode(
#    f"{os.environ.get('LANGFUSE_PUBLIC_KEY')}:{os.environ.get('LANGFUSE_SECRET_KEY')}".encode()
#).decode()

# Configure OpenTelemetry endpoint & headers
#os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = os.environ.get("LANGFUSE_HOST") + "/api/public/otel"
#os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"



# Your openai key
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

✅ Configuración de Nebius AI completada
✅ Modelo: meta-llama/Llama-3.3-70B-Instruct
✅ Langfuse Host: https://cloud.langfuse.com


## 3. Instrumenting the Agent

Pydantic Logfire offers an instrumentation for the OpenAi Agent SDK. We use this to send traces to Langfuse.

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
import logfire

# Configure logfire instrumentation.
logfire.configure(
    service_name='my_agent_service',
    send_to_logfire=False,
)
# This method automatically patches the OpenAI Agents SDK to send logs via OTLP to Langfuse.
logfire.instrument_openai_agents()

Now initialize the Langfuse client. `get_client()` initializes the Langfuse client using the credentials provided in the environment variables.

In [5]:
from langfuse import get_client

langfuse = get_client()

# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

Langfuse client is authenticated and ready!


## 4. Hello World Example

Below we create an **OpenAI Agent** that always replies in **haiku** form. We run it with `Runner.run` and print the final output.


In [ ]:
import asyncio
from agents import Agent, Runner

async def main():
    agent = Agent(
        name="Assistant",
        instructions="You only respond in haikus.",
        model="gpt-4.1-mini"
    )

    result = await Runner.run(agent, "Tell me about recursion in programming.")
    print(result.final_output)

loop = asyncio.get_running_loop()
await loop.create_task(main())

12:43:53.168 OpenAI Agents trace: Agent workflow
12:43:53.172   Agent run: 'Assistant'
12:43:53.174     Chat completion with 'meta-llama/Llama-3.3-70B-Instruct' [LLM]
Functions call self
Echoes of code repeat deep
Solving with each step


In [7]:
# langfuse.flush()

![Example trace in Langfuse](https://langfuse.com/images/cookbook/integration_openai-agents/openai-agent-example-trace.png)

**Example**: [Langfuse Trace](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/019593c7330da67c08219bd1c75b7a6d?timestamp=2025-03-14T08%3A31%3A00.365Z&observation=81e525d819153eed)

Clicking the link above (or your own project link) lets you view all sub-spans, token usage, latencies, etc., for debugging or optimization.

## 5. Multi-agent Handoff Example

Here we create:
- A **Spanish agent** that responds only in Spanish.
- An **English agent** that responds only in English.
- A **Triage agent** that routes the request to the correct agent based on the input language.

Any calls or handoffs are captured as part of the trace. That way, you can see which sub-agent or tool was used, as well as the final result.

In [ ]:
from agents import Agent, Runner
import asyncio

spanish_agent = Agent(
    name="Spanish agent",
    instructions="You only speak Spanish.",
     model="gpt-4.1-mini",
)

english_agent = Agent(
    name="English agent",
    instructions="You only speak English",
    model="gpt-4.1-mini",
)

triage_agent = Agent(
    name="Triage agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[spanish_agent, english_agent],
    model="gpt-4.1-mini"
)

result = await Runner.run(triage_agent, input="Hola, ¿cómo estás?")
print(result.final_output)

12:43:54.680 OpenAI Agents trace: Agent workflow
12:43:54.680   Agent run: 'Triage agent'
12:43:54.681     Chat completion with 'meta-llama/Llama-3.3-70B-Instruct' [LLM]


[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Your authentication token is not from a valid issuer.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_issuer"
  }
}


12:43:55.890     Handoff: Triage agent → None
12:43:55.891   Agent run: 'Spanish agent'
12:43:55.892     Chat completion with 'meta-llama/Llama-3.3-70B-Instruct' [LLM]
¡Hola! ¿En qué puedo ayudarte hoy?


![Example trace in Langfuse](https://langfuse.com/images/cookbook/integration_openai-agents/openai-agent-example-trace-handoff.png)

**Example**: [Langfuse Trace](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/019593c74429a6d0489e9259703a1148?timestamp=2025-03-14T08%3A31%3A04.745Z&observation=e83609282c443b0d)

## 6. Functions Example

The OpenAI Agents SDK allows the agent to call Python functions. With Langfuse instrumentation, you can see which **functions** are called, their arguments, and the return values. Here we define a simple function `get_weather(city: str)` and add it as a tool.


In [ ]:
import asyncio
from agents import Agent, Runner, function_tool

# Example function tool.
@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny."

agent = Agent(
    name="Hello world",
    instructions="You are a helpful agent.",
    tools=[get_weather],
)

async def main():
    result = await Runner.run(agent, input="What's the weather in Tokyo?")
    print(result.final_output)

loop = asyncio.get_running_loop()
await loop.create_task(main())

12:43:56.507 OpenAI Agents trace: Agent workflow
12:43:56.508   Agent run: 'Hello world'
12:43:56.509     Chat completion with 'meta-llama/Llama-3.3-70B-Instruct' [LLM]
12:43:57.483     Function: get_weather
12:43:57.485     Chat completion with 'meta-llama/Llama-3.3-70B-Instruct' [LLM]
I am not able to execute this request. Please try something else.


[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Your authentication token is not from a valid issuer.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_issuer"
  }
}


![Example trace in Langfuse](https://langfuse.com/images/cookbook/integration_openai-agents/openai-agent-example-trace-function.png)

**Example**: [Langfuse Trace](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/019593c74a162f93387d9261b01f9ca9?timestamp=2025-03-14T08%3A31%3A06.262Z&observation=0e2988966786cdf4)

When viewing the trace, you’ll see a span capturing the function call `get_weather` and the arguments passed.

## Resources

- [Example notebook on evaluating agents](https://langfuse.com/guides/cookbook/example_evaluating_openai_agents) with Langfuse.

<!-- MARKDOWN_COMPONENT name: "LearnMore" path: "@/components-mdx/integration-learn-more.mdx" -->